<a href="https://colab.research.google.com/github/dhruthick/cse256project/blob/main/LSTM-trackemb-rprc0.08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM for playlist continuation with just tracks


## Imports

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from gensim.models import Word2Vec
from tqdm.notebook import tqdm
import math

In [2]:
import torch

# Confirm that the GPU is detected

if torch.cuda.is_available():
  # Get the GPU device name.
  device_name = torch.cuda.get_device_name()
  n_gpu = torch.cuda.device_count()
  print(f"Found device: {device_name}, n_gpu: {n_gpu}")
  device = torch.device("cuda")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Found device: Tesla T4, n_gpu: 1


## Read data

In [3]:
data_path='/content/drive/MyDrive/cse256/project/data/'
interactions=pd.read_csv(data_path+'interactions_train.csv')
tracks=interactions[['track_uri','artist_uri','album_uri']].drop_duplicates()

## Some necessary data structures and utilities 

In [4]:
track_uri2id={}
track_id2uri={}

track_id=0
for index,row in tracks.iterrows():
  track_id+=1
  track_uri2id[row[0]]=track_id
  track_id2uri[track_id]=row[0]

In [5]:
playlists=defaultdict(list)
for pid, playlist_tracks in interactions.groupby('pid'):
  if len(playlist_tracks)>1:
    playlists[pid].extend([track_uri2id[t] for t in playlist_tracks.track_uri.values.tolist()])

In [6]:
def playlist_tracks_sentence(track_ids):
  return [str(i) for i in track_ids]

In [7]:
sentences=[playlist_tracks_sentence(playlists[p]) for p in playlists]
vocab_size=len(track_uri2id)+1

## Word embeddings for tracks with Word2Vec

In [8]:
track_emb_model=Word2Vec(sentences,vector_size=100,min_count=0)

In [9]:
track_embeddings=torch.zeros(vocab_size+1,100)
for track_id in track_id2uri.keys():
  track_embeddings[track_id]=torch.FloatTensor(track_emb_model.wv[str(track_id)])

<ipython-input-9-04c72592a695>:3: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  track_embeddings[track_id]=torch.FloatTensor(track_emb_model.wv[str(track_id)])


## Pre-processing for training

In [10]:
sequences=[]
for pid, tracks in playlists.items():
  for i in range(2,len(tracks)+1):
    sequences.append(tracks[:i])
X_train,y_train=[],[]
for s in sequences:
  X_train.append(torch.tensor(s[:-1],dtype=torch.int32))
  y_train.append(s[-1])
X_train=torch.nn.utils.rnn.pad_sequence(X_train,batch_first=True)

## Model definition 

In [11]:
class LSTMrec(torch.nn.Module):

  def __init__(self, vocab_size, emb_matrix, hidden_dim, num_layers, dropout_rate):
    super().__init__()
    self.num_layers=num_layers
    self.hidden_dim=hidden_dim
    self.embedding_dim=emb_matrix.shape[1]
    self.embedding = torch.nn.Embedding.from_pretrained(torch.FloatTensor(emb_matrix),
                                                  freeze=True)
    self.lstm=torch.nn.LSTM(self.embedding_dim,
                      hidden_dim,
                      num_layers=num_layers,
                      dropout=dropout_rate, batch_first=True)
    # self.dropout=torch.nn.Dropout(dropout_rate)
    self.fc=torch.nn.Linear(hidden_dim,vocab_size)

  def forward(self, input, hidden):
    embedding=self.embedding(input)
    output,hidden=self.lstm(embedding, hidden)
    prediction=self.fc(output[:,-1,:])
    return prediction,hidden

  def init_hidden(self, batch_size,device):
    hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
    cell = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
    return hidden, cell

  def detach_hidden(self, hidden):
    hidden, cell = hidden
    hidden = hidden.detach()
    cell = cell.detach()
    return hidden, cell

## Model initialization and training

In [12]:
model = LSTMrec(vocab_size, track_embeddings, hidden_dim=50, num_layers=1,
                dropout_rate=0.1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Function to calculate number of correct predictions

In [13]:
def calculate_correct(logits, labels):
  logits = logits.detach().cpu().numpy()
  label_ids = labels.to('cpu').numpy()
  # Calculate the number of correctly labeled examples in batch
  pred_flat = np.argmax(logits, axis=1).flatten()
  labels_flat = label_ids.flatten()
  num_correct = np.sum(pred_flat == labels_flat)
  return num_correct

Training

In [14]:
def train(model, X_train, y_train, optimizer, criterion, batch_size,epochs):
    for epoch in tqdm(range(1,epochs+1)):

      epoch_loss = 0
      model.train()
      num_batches = int(len(X_train)/batch_size)+1
      hidden = model.init_hidden(batch_size,device)
      total_correct=0
      for i in tqdm(range(num_batches)):
          optimizer.zero_grad()
          end_index = min(batch_size * (i+1), len(X_train))
          hidden = model.detach_hidden(hidden)
          X_batch = X_train[i*batch_size:end_index]
          X_batch=X_batch.to(device)
          if len(X_batch)!=batch_size:
              hidden = model.init_hidden(len(X_batch),device)
          if len(X_batch) == 0: continue
          y_batch = y_train[i*batch_size:end_index]
          y_batch=y_batch.to(device)
          
          prediction, hidden = model(X_batch, hidden)
          total_correct+=calculate_correct(prediction, y_batch)
          
          # print(total_correct)
          loss = criterion(prediction, y_batch)
          loss.backward()
          # torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
          optimizer.step()
          epoch_loss += loss.item()
    
      print(f'Epoch {epoch}: loss = {epoch_loss}, train_acc = {total_correct/len(X_train)}')
    return model

In [15]:
model=train(model,X_train,torch.tensor(y_train),optimizer, criterion, batch_size=64,epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/5356 [00:00<?, ?it/s]

Epoch 1: loss = 64545.3862695694, train_acc = 0.0003588893687048471


  0%|          | 0/5356 [00:00<?, ?it/s]

Epoch 2: loss = 59580.053873062134, train_acc = 0.0004172453636162043


...

Epoch 8: loss = 62346.94559574127, train_acc = 0.0002771909758289469


  0%|          | 0/5356 [00:00<?, ?it/s]

Epoch 9: loss = 62255.565485954285, train_acc = 0.0002946977743023541


  0%|          | 0/5356 [00:00<?, ?it/s]

Epoch 10: loss = 61942.138956069946, train_acc = 0.0002917799745567862


## Evaluation on validataion set

In [16]:
val=pd.read_csv(data_path+'interactions_val.csv')

In [17]:
def evaluate_playlist_rec(pid,N=500):
  tracksInPlaylist=playlists[pid]
  relevantTracks=set(val[val['pid']==pid].track_uri.values)
  tracksInPlaylist=torch.tensor(tracksInPlaylist).to(device).unsqueeze(0)
  prediction, hidden=model(tracksInPlaylist,model.init_hidden(1,device))
  prediction = prediction[0].to('cpu')
  scores = [(float(prediction[i]), track_id2uri[i] ) for i in range(1, len(prediction)) if i not in playlists[pid]]
  scores.sort(reverse=True)
  recommendedTracks=set([t[1] for t in scores[:N]])
  rprc=len(recommendedTracks.intersection(relevantTracks))/len(relevantTracks)
  dcg=0
  for i in range(len(scores)):
    if scores[i][1] in relevantTracks:
      dcg+=math.log(2)/math.log(i+2)
  ndcg=dcg/len(relevantTracks)
  rec_click=N/10+1
  for i in range(0,50):
    recommendedTracks=set([t[1] for t in scores[i*10:(i*10+10)]])
    if len(recommendedTracks.intersection(relevantTracks))>0:
      rec_click=i+1
      break
  return rprc,ndcg,rec_click

In [18]:
val_pids=np.unique(val.pid.values)
rprcs,ndcgs,rec_clicks=[],[],[]
for pid in tqdm(val_pids):
  rprc,ndcg,rec_click=evaluate_playlist_rec(pid,N=500)
  rprcs.append(rprc)
  ndcgs.append(ndcg)
  rec_clicks.append(rec_click)

print(f'\nAverage R-Precision: {np.average(rprcs)}')
print(f'Average NDCG: {np.average(ndcgs)}')
print(f'Average Recommendation Clicks: {np.average(rec_clicks)}')

  0%|          | 0/7476 [00:00<?, ?it/s]


Average R-Precision: 0.0857724021327227
Average NDCG: 0.0665422029802895
Average Recommendation Clicks: 37.512841091492774
